# Norepinephrine-to-fluid ratio (NFR) in septic patients

## Import libraries etc.

In [1]:
# Import Pandas, Numpy and SQL-Connection
import numpy as np
import pandas as pd
import sqlalchemy
import psycopg2

# Import ML- and statistic tools
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from math import sqrt
import scipy.stats as stats

# Import plotting tools
import matplotlib.pyplot as plt
import seaborn as sns

# Import AmsterdamUMCdb
import amsterdamumcdb as adb

# Import local code
import adbcalc as ac

# Import graphviz
import graphviz
from graphviz import Digraph

# Import stats
from scipy.stats import pointbiserialr

### Connection

In [2]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('config.ini'):
    config.read('config.ini')
else:
    config.read('../config.SAMPLE.ini')

#Open a connection to the postgres database:
pg_con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
pg_con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
pg_con.set_session(autocommit=True)

cursor = pg_con.cursor()
cursor.execute('SET SCHEMA \'amsterdamumcdb\''); #set search_path to amsterdamumcdb schema

# create SQLAlchemy engine for official pandas database support
engine = sqlalchemy.create_engine('postgresql+psycopg2://', creator=lambda: pg_con)
con = engine.connect()

NameError: name 'sqlalchemy' is not defined

### ROC-AUC Confidence interval calculation

In [ ]:
def roc_auc_ci(
        y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (round(lower, 2), round(upper, 2))

## Data preparation

### Define observation period in ms

In [ ]:
fluid_period = 3 * 24 * 60 * 60 * 1000

### Extract septic patients

In [ ]:
sepsis_df = adb.get_sepsis_patients(con)

### Execute SQL queries

In [ ]:
sepsis_ids = tuple(sepsis_df['admissionid'])

In [ ]:
# Intake of crystalloids
query = open('./sql/crystalloids.sql', 'r')
crystalloids = pd.read_sql_query(query.read(), con, params={"fluid_period": fluid_period, "sepsis_ids": sepsis_ids})

In [ ]:
# Intake of colloids
query = open('./sql/colloids.sql', 'r')
colloids = pd.read_sql_query(query.read(), con, params={"fluid_period": fluid_period, "sepsis_ids": sepsis_ids})

In [ ]:
# Intake of blood products
query = open('./sql/bloodproducts.sql', 'r')
bloodproducts = pd.read_sql_query(query.read(), con, params={"fluid_period": fluid_period, "sepsis_ids": sepsis_ids})

In [ ]:
# Intake of norepinephrine
query = open('./sql/norepinephrine.sql', 'r')
norepinephrine = pd.read_sql_query(query.read(), con, params={"fluid_period": fluid_period, "sepsis_ids": sepsis_ids})

In [ ]:
# Patient demographics and other information
query = open('./sql/patients.sql', 'r')
patients = pd.read_sql_query(query.read(), con, params={"sepsis_ids": sepsis_ids})

### Calculate cumulative doses

In [ ]:
# Calculate the norepinephrine dose in microgram (mcg) for every episode of norepinephrine infusion
norepi_mcg = [ac.calculate_dose(norepinephrine['start'][i], \
                                          norepinephrine['stop'][i], \
                                          fluid_period, \
                                          norepinephrine['dose'][i], \
                                          1)
              for i in norepinephrine.index]

In [ ]:
# Calculate the crystalloid volume in mililiters (ml) for every episode of crystalloid infusion
crystalloids_ml = [ac.calculate_dose(crystalloids['start'][i], \
                                          crystalloids['stop'][i], \
                                          fluid_period, \
                                          crystalloids['dose'][i], \
                                          2)
              for i in crystalloids.index]

In [ ]:
# Calculate the colloid volume in mililiters (ml) for every episode of colloid infusion
colloids_ml = [ac.calculate_dose(colloids['start'][i], \
                                          colloids['stop'][i], \
                                          fluid_period, \
                                          colloids['dose'][i], \
                                          2)
              for i in colloids.index]

In [ ]:
# Calculate the bloodproducts volume in mililiters (ml) for every episode of bloodproduct infusion
bloodproducts_ml = [ac.calculate_dose(bloodproducts['start'][i], \
                                          bloodproducts['stop'][i], \
                                          fluid_period, \
                                          bloodproducts['dose'][i], \
                                          2)
              for i in bloodproducts.index]

In [ ]:
# Add columns with cumulative dose
norepinephrine['norepi_mcg'] = norepi_mcg
crystalloids['crystalloids_ml'] = crystalloids_ml
colloids['colloids_ml'] = colloids_ml
bloodproducts['bloodproducts_ml'] = bloodproducts_ml

In [ ]:
# Sum up all intervals
norepinephrine_sum = norepinephrine[['admissionid', 'norepi_mcg']].groupby(['admissionid']).sum()
norepinephrine_max = norepinephrine[['admissionid', 'dose', 'stop']].loc[norepinephrine.groupby(['admissionid'])['dose'].idxmax()].set_index('admissionid').rename({'dose': 'norepi_max_dose', 'stop': 'norepi_max_dose_stop'}, axis='columns')
crystalloids_sum = crystalloids[['admissionid', 'crystalloids_ml']].groupby(['admissionid']).sum()
colloids_sum = colloids[['admissionid', 'colloids_ml']].groupby(['admissionid']).sum()
bloodproducts_sum = bloodproducts[['admissionid', 'bloodproducts_ml']].groupby(['admissionid']).sum()

### Merge Pandas DataFrames

In [ ]:
patients['dateofdeath'] = patients['dateofdeath'].fillna(0)
patients['died'] = np.where(patients['dateofdeath'] != 0 , 1, 0)
patients['died07'] = np.where((patients['dateofdeath'] != 0) & (patients['dateofdeath'] < 604800000), 1, 0)
patients['died28'] = np.where((patients['dateofdeath'] != 0) & (patients['dateofdeath'] < 2419200000), 1, 0)

In [ ]:
# Merge DataFrames
intake = pd.merge(patients, crystalloids_sum, how = 'left', on = 'admissionid') \
    .merge(colloids_sum, on = 'admissionid', how = 'left') \
    .merge(bloodproducts_sum, on = 'admissionid', how = 'left') \
    .merge(norepinephrine_sum, on = 'admissionid', how = 'left') \
    .merge(norepinephrine_max, on = 'admissionid', how = 'left')

### Fill NA and drop out patients w/o norepinephrine or fluids

In [ ]:
# Fill NULL-values with 0
intake['died'] = intake['died'].fillna(0)
intake['crystalloids_ml'] = intake['crystalloids_ml'].fillna(0)
intake['colloids_ml'] = intake['colloids_ml'].fillna(0)
intake['bloodproducts_ml'] = intake['bloodproducts_ml'].fillna(0)
intake['norepi_mcg'] = intake['norepi_mcg'].fillna(0)
intake['total_fluids'] = intake['crystalloids_ml'] + intake['colloids_ml'] + intake['bloodproducts_ml']
intake['nf_ratio'] = intake['norepi_mcg'] / intake['total_fluids']

In [ ]:
# Test if pandas merge is plausible
print('Sepsis: ', len(patients))
print('Crystalloids rows: ', len(crystalloids))
print('Colloids rows: ', len(colloids))
print('Blood products rows: ', len(bloodproducts))
print('Norepinephrine rows: ', len(norepinephrine))
print('Merged DataFrame rows: ', len(intake))

In [ ]:
# Drop rows without norepinephrine and without fluids
intake = intake.drop(intake[(intake.norepi_mcg == 0) | \
                            (intake.norepi_max_dose == 0) | \
                            (intake.total_fluids == 0) | \
                            (intake.nf_ratio == 0)].index) 

## Demographics

In [ ]:
intake[['admissionid', 'agegroup']].groupby(['agegroup']).count()

In [ ]:
print('Male: ', len(intake[intake['gender'] == 'Man']))
print('Female: ', len(intake[intake['gender'] == 'Vrouw']))
print('Unknown: ', len(intake[intake['gender'] != 'Vrouw']))

## Statistics

### Overall descriptive stats

In [ ]:
# Select 
intake_survived = intake[['died', 'dateofdeath', 'crystalloids_ml', 'colloids_ml', 'bloodproducts_ml', 'norepi_mcg', 'norepi_max_dose', 'total_fluids', 'nf_ratio']][intake['died'] == 0]
intake_died = intake[['died', 'dateofdeath', 'crystalloids_ml', 'colloids_ml', 'bloodproducts_ml', 'norepi_mcg', 'norepi_max_dose', 'total_fluids', 'nf_ratio']][intake['died'] == 1]
intake_died07 = intake[['died07', 'dateofdeath', 'crystalloids_ml', 'colloids_ml', 'bloodproducts_ml', 'norepi_mcg', 'norepi_max_dose', 'total_fluids', 'nf_ratio']][intake['died07'] == 1]
intake_died28 = intake[['died28', 'dateofdeath', 'crystalloids_ml', 'colloids_ml', 'bloodproducts_ml', 'norepi_mcg', 'norepi_max_dose', 'total_fluids', 'nf_ratio']][intake['died28'] == 1]

In [ ]:
#intake_survived.describe()

In [ ]:
#intake_survived.median()

In [ ]:
#intake_died.describe()

In [ ]:
#intake_died07.describe()

In [ ]:
#intake_died28.describe()

In [ ]:
#intake['nf_ratio'].describe()

In [ ]:
#intake['norepi_max_dose'].describe()

### NFR descriptive stats

In [ ]:
#intake['nf_ratio'][intake['died07'] ==0].describe()

In [ ]:
#intake['nf_ratio'][intake['died07'] ==1].describe()

### Point-biserial correlation

In [ ]:
pbc07 = pointbiserialr(intake['died07'], intake['nf_ratio'])
pbc28 = pointbiserialr(intake['died28'], intake['nf_ratio'])
pbc07_norepi = pointbiserialr(intake['died07'], intake['norepi_mcg'])
pbc28_norepi = pointbiserialr(intake['died28'], intake['norepi_mcg'])
pbc07_fluids = pointbiserialr(intake['died07'], intake['total_fluids'])
pbc28_fluids = pointbiserialr(intake['died28'], intake['total_fluids'])

print('NFR 7-day:', pbc07)
print('NFR 28-day:', pbc28)
print('Norepinpehrine 7-day:', pbc07_norepi)
print('Norepinpehrine 28-day:', pbc28_norepi)
print('Fluids 7-day:', pbc07_fluids)
print('Fluids 28-day:', pbc28_fluids)

### ROC AUC and Youden-Index

In [ ]:
# Area under curve score is calculated using sklearn.metrics
# Youden index is calculated as the maximal value of true positive rate (tpr) - false positive rate (fpr) 
# (= 1 - specificity)

# NFR
auc07 = roc_auc_score(intake['died07'], intake['nf_ratio'])
fpr07, tpr07, thresholds07 = roc_curve(intake['died07'], intake['nf_ratio'])
optimal_cutoff_pos07 = np.argmax(tpr07 - fpr07)
optimal_cutoff07 = thresholds07[optimal_cutoff_pos07]
youden_index07 = (tpr07 - fpr07)[optimal_cutoff_pos07]
auroc_ci07 = roc_auc_ci(intake['died07'], intake['nf_ratio'])
sensitivity07 = tpr07[optimal_cutoff_pos07]
specificity07 = 1-fpr07[optimal_cutoff_pos07]

auc28 = roc_auc_score(intake['died28'], intake['nf_ratio'])
fpr28, tpr28, thresholds28 = roc_curve(intake['died28'], intake['nf_ratio'])
optimal_cutoff_pos28 = np.argmax(tpr28 - fpr28)
optimal_cutoff28 = thresholds28[optimal_cutoff_pos28]
youden_index28 = (tpr28 - fpr28)[optimal_cutoff_pos28]
auroc_ci28 = roc_auc_ci(intake['died28'], intake['nf_ratio'])
sensitivity28 = tpr28[optimal_cutoff_pos28]
specificity28 = 1-fpr28[optimal_cutoff_pos28]

# Norepinephrine
auc07_norepi = roc_auc_score(intake['died07'], intake['norepi_mcg'])
fpr07_norepi, tpr07_norepi, thresholds07_norepi = roc_curve(intake['died07'], intake['norepi_mcg'])
optimal_cutoff_pos07_norepi = np.argmax(tpr07_norepi - fpr07_norepi)
optimal_cutoff07_norepi = thresholds07_norepi[optimal_cutoff_pos07_norepi]
youden_index07_norepi = (tpr07_norepi - fpr07_norepi)[optimal_cutoff_pos07_norepi]
auroc_ci07_norepi = roc_auc_ci(intake['died07'], intake['norepi_mcg'])
sensitivity07_norepi = tpr07_norepi[optimal_cutoff_pos07_norepi]
specificity07_norepi = 1-fpr07_norepi[optimal_cutoff_pos07_norepi]

auc28_norepi = roc_auc_score(intake['died28'], intake['norepi_mcg'])
fpr28_norepi, tpr28_norepi, thresholds28_norepi = roc_curve(intake['died28'], intake['norepi_mcg'])
optimal_cutoff_pos28_norepi = np.argmax(tpr28_norepi - fpr28_norepi)
optimal_cutoff28_norepi = thresholds28_norepi[optimal_cutoff_pos28_norepi]
youden_index28_norepi = (tpr28_norepi - fpr28_norepi)[optimal_cutoff_pos28_norepi]
auroc_ci28_norepi = roc_auc_ci(intake['died28'], intake['norepi_mcg'])
sensitivity28_norepi = tpr28_norepi[optimal_cutoff_pos28_norepi]
specificity28_norepi = 1-fpr28_norepi[optimal_cutoff_pos28_norepi]

# Fluid intake
auc07_fluids = roc_auc_score(intake['died07'], intake['total_fluids'])
fpr07_fluids, tpr07_fluids, thresholds07_fluids = roc_curve(intake['died07'], intake['total_fluids'])
optimal_cutoff_pos07_fluids = np.argmax(tpr07_fluids - fpr07_fluids)
optimal_cutoff07_fluids = thresholds07_fluids[optimal_cutoff_pos07_fluids]
youden_index07_fluids = (tpr07_fluids - fpr07_fluids)[optimal_cutoff_pos07_fluids]
auroc_ci07_fluids = roc_auc_ci(intake['died07'], intake['total_fluids'])
sensitivity07_fluids = tpr07_fluids[optimal_cutoff_pos07_fluids]
specificity07_fluids = 1-fpr07_fluids[optimal_cutoff_pos07_fluids]

auc28_fluids = roc_auc_score(intake['died28'], intake['total_fluids'])
fpr28_fluids, tpr28_fluids, thresholds28_fluids = roc_curve(intake['died28'], intake['total_fluids'])
optimal_cutoff_pos28_fluids = np.argmax(tpr28_fluids - fpr28_fluids)
optimal_cutoff28_fluids = thresholds28_fluids[optimal_cutoff_pos28_fluids]
youden_index28_fluids = (tpr28_fluids - fpr28_fluids)[optimal_cutoff_pos28_fluids]
auroc_ci28_fluids = roc_auc_ci(intake['died28'], intake['total_fluids'])
sensitivity28_fluids = tpr28_fluids[optimal_cutoff_pos28_fluids]
specificity28_fluids = 1-fpr28_fluids[optimal_cutoff_pos28_fluids]

print('Observation period:', fluid_period / (24 * 60 * 60 * 1000), 'days')
print()
print('NFR AUROC 7-day mortality:', round(auc07, 2))
print('NFR AUROC 7-day mortality confidence interval:', auroc_ci07)
print('NFR Youden-Index 7-day mortality:' , round(youden_index07, 2))
print('NFR Optimal cut-off 7-day mortality:', round(optimal_cutoff07, 2))
print('NFR Sensitivity 7-day mortality:', round(sensitivity07, 2))
print('NFR Specificity 7-day mortality:', round(specificity07, 2))
print()
print('NFR AUROC 28-day mortality:', round(auc28, 2))
print('NFR AUC-ROC 28-day mortality confidence interval:', auroc_ci28)
print('NFR Youden-Index 28-day mortality:' , round(youden_index28, 2))
print('NFR Optimal cut-off 28-day mortality:', round(optimal_cutoff28, 2))
print('NFR Sensitivity 28-day mortality:', round(sensitivity28, 2))
print('NFR Specificity 28-day mortality:', round(specificity28, 2))
print()
print('Norepinephrine AUROC 7-day mortality:', round(auc07_norepi, 2))
print('Norepinephrine AUROC 7-day mortality confidence interval:', auroc_ci07_norepi)
print('Norepinephrine Youden-Index 7-day mortality:' , round(youden_index07_norepi, 2))
print('Norepinephrine optimal cut-off 7-day mortality:', round(optimal_cutoff07_norepi, 2))
print('Norepinephrine Sensitivity 7-day mortality:', round(sensitivity07_norepi, 2))
print('Norepinephrine Specificity 7-day mortality:', round(specificity07_norepi, 2))
print()
print('Norepinephrine AUROC 28-day mortality:', round(auc28_norepi, 2))
print('Norepinpehrine AUROC 28-day mortality confidence interval:', auroc_ci28_norepi)
print('Norepinheprine Youden-Index 28-day mortality:' , round(youden_index28_norepi, 2))
print('Norepinpehrine Optimal cut-off 28-day mortality:', round(optimal_cutoff28_norepi, 2))
print('Norepinephrine Sensitivity 28-day mortality:', round(sensitivity28_norepi, 2))
print('Norepinephrine Specificity 28-day mortality:', round(specificity28_norepi, 2))
print()
print('Fluids AUROC 7-day mortality:', round(auc07_fluids, 2))
print('Fluids AUROC 7-day mortality confidence interval:', auroc_ci07_fluids)
print('Fluids Youden-Index 7-day mortality:' , round(youden_index07_fluids, 2))
print('Fluids optimal cut-off 7-day mortality:', round(optimal_cutoff07_fluids, 2))
print('Fluids Sensitivity 7-day mortality:', round(sensitivity07_fluids, 2))
print('Fluids Specificity 7-day mortality:', round(specificity07_fluids, 2))
print()
print('Fluids AUROC 28-day mortality:', round(auc28_fluids, 2))
print('Fluids AUROC 28-day mortality confidence interval:', auroc_ci28_fluids)
print('Fluids Youden-Index 28-day mortality:' , round(youden_index28_fluids, 2))
print('Fluids optimal cut-off 28-day mortality:', round(optimal_cutoff28_fluids, 2))
print('Fluids Sensitivity 28-day mortality:', round(sensitivity28_norepi, 2))
print('Fluids Specificity 28-day mortality:', round(specificity28_norepi, 2))

### Odds ratio

In [ ]:
# Create two-dimensional contingency table
ct07 = pd.crosstab(index = intake.died07, columns = intake.nf_ratio >= optimal_cutoff07)
ct28 = pd.crosstab(index = intake.died28, columns = intake.nf_ratio >= optimal_cutoff28)
print(ct07)

In [ ]:
# Calculate odds-ratio
oddsratio07, pvalue07 = stats.fisher_exact(ct07)
oddsratio28, pvalue28 = stats.fisher_exact(ct28)
print("7-day mortality Odds-Ratio:", round(oddsratio07, 2), "p-Value:", pvalue07)
print("28-day mortality Odds-Ratio:", round(oddsratio28, 2), "p-Value:", pvalue28)

### Relative risk

In [ ]:
rel_risk_nfr07 = round((ct07[True][1] / (ct07[True][1] + ct07[True][0])) \
/ (ct07[False][1] / (ct07[False][1] + ct07[False][0])), 2)

rel_risk_nfr28 = round((ct28[True][1] / (ct28[True][1] + ct28[True][0])) \
/ (ct28[False][1] / (ct28[False][1] + ct28[False][0])), 2)

print('Relative risk 7-day mortality:', round(rel_risk_nfr07, 2))
print('Relative risk 28-day mortality:', round(rel_risk_nfr28, 2))

## Plots

### ROC-Curve

In [ ]:
sns.set_theme()
sns.set_style('whitegrid')

In [ ]:
fpr07_df = pd.DataFrame({'fpr07': fpr07}).melt(var_name = 'fpr', value_name = 'fpr_val')
tpr07_df = pd.DataFrame({'tpr07': tpr07}).melt(var_name = 'tpr', value_name = 'tpr_val')
fpr28_df = pd.DataFrame({'fpr28': fpr28}).melt(var_name = 'fpr', value_name = 'fpr_val')
tpr28_df = pd.DataFrame({'tpr28': tpr28}).melt(var_name = 'tpr', value_name = 'tpr_val')

In [ ]:
fprdf = fpr07_df.append(fpr28_df, ignore_index=True)
tprdf = tpr07_df.append(tpr28_df, ignore_index=True)
auc_values_lf = fprdf.join(tprdf)

In [ ]:
sns.set(rc={'axes.facecolor':'#eee8d5', 'figure.facecolor':'#eee8d5'})
ax = sns.lineplot(x = 'fpr_val', 
                  y = 'tpr_val', 
                  hue = 'fpr', 
                  data = auc_values_lf,
                  style = "fpr",
                  palette = ['#268bd2', '#b58900'])
ax.set_aspect("equal")
ax.set_xlabel('1 - Specificity', color = '#586e76')
ax.set_ylabel('Sensitivity', color = '#586e76')
#ax.set(xlabel = '1 - Specificity', ylabel = 'Sensitivity')
ax.legend(title='ROC-curve', loc='lower right', labels=['7-day mortality', '28-day mortality'])

# Save plot
plt.savefig('img/auroc.png', dpi=300)

### Distribution

In [ ]:
sns.histplot(x = 'nf_ratio', data = intake, hue='died07', log_scale=True)
plt.axvline(optimal_cutoff07)

In [ ]:
sns.displot(x = 'nf_ratio', data = intake, hue='died07', kind='kde', log_scale=True, fill=True)
plt.axvline(optimal_cutoff07)

In [ ]:
sns.displot(x = 'nf_ratio', data = intake, hue='died28', kind='kde', log_scale=True, fill=True)
plt.axvline(optimal_cutoff28)

In [ ]:
sns.displot(x = 'nf_ratio', data = intake, hue='died07', log_scale=True, kind='ecdf')

### Boxplot

In [ ]:
lf07 = intake[['died07']].melt(value_name='died').join(intake[['nf_ratio']].melt(var_name='nfr', value_name='nfratio'))
lf28 = intake[['died28']].melt(value_name='died').join(intake[['nf_ratio']].melt(var_name='nfr', value_name='nfratio'))

In [ ]:
boxplot_lf = lf07.append(lf28, ignore_index=True)

In [ ]:
boxplot_lf.head()

In [ ]:
sns.set(rc={'axes.facecolor':'#eee8d5', 'figure.facecolor':'#eee8d5'})
ax = sns.boxplot(x = 'died', 
                 y = 'nfratio', 
                 hue = 'variable', 
                 data = boxplot_lf, 
                 showfliers = False, 
                 palette = ['#268bd2', '#b58900'])
ax.set_xticklabels(['Survivors', 'Non-Survivors'], color = '#586e76')
#ax.set(xlabel = '', ylabel = 'Norepinephrine-to-fluid ratio')
ax.set_xlabel('', color = '#586e76')
ax.set_ylabel('Norepinephrine-to-fluid ratio', color = '#586e76')
ax.legend(title="Observation time")
ax.legend_.texts[0].set_text('7 days')
ax.legend_.texts[1].set_text('28 days')

# Save plot
plt.savefig('img/boxplot_survival.png', dpi=300)

In [ ]:
ax = sns.boxplot(x = intake['died07'],y=intake['nf_ratio'], showfliers=False)
ax.set_xticklabels(['Survivors', 'Non-Survivors'])
ax.set(xlabel = '', ylabel = 'Norepinephrine-fluid-ratio')